# Analyze links of invectio and solver outputs

## Set environment variables to access the datasets on Ceph

For more detail on the Operate First Ceph public bucket used here, visit https://www.operate-first.cloud/apps/content/odh/trino/access_public_bucket.html

In [ ]:
%env THOTH_CEPH_KEY_ID=LLEzCoxu7pvjzO4inoL8
%env THOTH_CEPH_SECRET_KEY=1HnDVoIS2jt3h3xEpgeQlCX5+FeOUH0wOrvWVvZP
%env THOTH_CEPH_BUCKET_PREFIX=thoth
%env THOTH_S3_ENDPOINT_URL=https://s3-openshift-storage.apps.smaug.na.operate-first.cloud
%env THOTH_CEPH_BUCKET=opf-datacatalog
%env THOTH_DEPLOYMENT_NAME=datasets

## Load modules

In [ ]:
from thoth.report_processing.components.solver import Solver
import pandas as pd

## Access data

In [ ]:
nonmatching_reports=Solver.aggregate_solver_results()

## Access all reports

In [ ]:
solver_reports_metadata = []
for solver_document in nonmatching_reports:
    solver_reports_metadata.append(
        Solver.extract_data_from_solver_metadata(nonmatching_reports[solver_document]["metadata"])
    )

solver_reports_metadata_df = pd.DataFrame(solver_reports_metadata)

solver_reports_metadata_df.head()

In [ ]:
solver_reports_extracted_data = []
solver_errors = []
for solver_document in nonmatching_reports:
    solver_report_extracted_data = Solver.extract_data_from_solver_metadata(
        nonmatching_reports[solver_document]["metadata"]
    )
    for k, v in nonmatching_reports[solver_document]["result"].items():
        solver_report_extracted_data[k] = v
        if k == "errors" and v:
            errors = Solver.extract_errors_from_solver_result(v)
            for error in errors:
                solver_errors.append(error)
    
    packages = Solver.extract_tree_from_solver_result(nonmatching_reports[solver_document]["result"])
    solver_report_extracted_data["packages"] = packages
    solver_reports_extracted_data.append(solver_report_extracted_data)

In [ ]:
solver_reports_metadata_df = pd.DataFrame(solver_reports_extracted_data)
solver_reports_metadata_df.head(10)

Compared to the previous the solver-dataset v1.0 in this we have extra output of:
- `platform` field 
- `packages` used in **tree** column


In [ ]:
solver_reports_metadata_df.loc[0]['tree'][0]['packages']

In [ ]:
solver_reports_metadata_df.loc[1]['tree'][0]['packages']

In [ ]:
solver_reports_metadata_df.loc[2]['tree'][0]['packages']

Point of interest is the discrepancy of aforementioned data and module names on pypi.org. For example:
- https://pypi.org/project/scikit-learn/
- https://pypi.org/project/ipython/ 
- https://pypi.org/project/python-dotenv/ 

## Invectio output

In order to link the solver output to invectio output let's analyze result of `invectio whatuses .` command on the following test python file

In [ ]:
from sklearn import linear_model as lm
import IPython
import dotenv
import plone.app.upgrade

plone_app_test = plone.app.upgrade.alias_module()
config = dotenv.dotenv_value(".env")
jobs = IPython.backgroundjobs.BackgroundJobManager()
lm.LinearRegression

result:

In [ ]:
    "tests/data/app_10_test.py": {
      "IPython": [
        "IPython.backgroundjobs.BackgroundJobManager"
      ],
      "dotenv": [
        "dotenv.dotenv_value"
      ],
      "plone": [
        "plone.app.upgrade.alias_module"
      ],
      "sklearn": [
        "sklearn.linear_model.LinearRegression"
      ]
    },

As we can see the name of modules will not match those from pypi: 
- ipython
- python-dotenv
- scikit-learn

## Possible link

To take a look on how invectio and solver datasets can be connected let's start with

In [ ]:
solver_reports_metadata_df.loc[0]['tree'][0]['importlib_metadata']['metadata']

Here we have both URL to the package in `Download-URL` and name in `Name`

In [ ]:
solver_reports_metadata_df.loc[0]['tree'][0]['importlib_metadata']['metadata']['Name']

Further, we can use function  `get_source_repos()` from [app.py](https://github.com/thoth-station/solver-project-url-job/blob/master/app.py) from [solver-project-url-job repository](https://github.com/thoth-station/solver-project-url-job) to match the output from `invectio whatuses .` to package name in `solver_reports_metadata_df.loc[0]['tree'][0]['packages']`. 
And match it in turn to `solver_reports_metadata_df.loc[0]['tree'][0]['importlib_metadata']['metadata']['Name']`
Refer below.

    "tests/data/app_10_test.py": {
      "IPython": [
        "IPython.backgroundjobs.BackgroundJobManager"
      ],
      "dotenv": [
        "dotenv.dotenv_value"
      ],
      "plone": [
        "plone.app.upgrade.alias_module"
      ],
      "sklearn": [
        "sklearn.linear_model.LinearRegression"
      ]
    },

In [ ]:
solver_reports_metadata_df.loc[0]['tree'][0]['packages']

In [ ]:
solver_reports_metadata_df.loc[0]['tree'][0]['importlib_metadata']['metadata']['Name']